<a href="https://colab.research.google.com/github/M-Ait/Projet8/blob/main/P8_Notebook_Linux_EMR_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Déployez un modèle dans le cloud

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1697546941232_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des informations sur la session en cours et liens vers Spark UI</u> :

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1697546941232_0001,pyspark,idle,Link,Link,None,✔


In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io 

from PIL import Image

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.functions import vector_to_array

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
PATH = 's3://fruits-images-projet8'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/PCA'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Reduced_Data: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://fruits-images-projet8
PATH_Data:   s3://fruits-images-projet8/Test
PATH_Reduced_Data: s3://fruits-images-projet8/PCA

# Traitement des données :

In [22]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://fruits-image...|2023-10-13 16:29:11|  7353|[FF D8 FF E0 00 1...|
|s3://fruits-image...|2023-10-13 16:29:14|  7350|[FF D8 FF E0 00 1...|
|s3://fruits-image...|2023-10-13 16:29:13|  7349|[FF D8 FF E0 00 1...|
|s3://fruits-image...|2023-10-13 16:29:12|  7348|[FF D8 FF E0 00 1...|
|s3://fruits-image...|2023-10-13 16:28:37|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [24]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------------+----------+
|path                                                    |label     |
+--------------------------------------------------------+----------+
|s3://fruits-images-projet8/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://fruits-images-projet8/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://fruits-images-projet8/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://fruits-images-projet8/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://fruits-images-projet8/Test/Watermelon/r_95_100.jpg |Watermelon|
+--------------------------------------------------------+----------+
only showing top 5 rows

None

In [31]:
images.toPandas().shape #22 688 images chargées

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(22688, 5)

In [54]:
pd_images = images.select('content').toPandas()
np.array(pd_images['content'][0]).shape  # taille d'une image

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(7353,)

# Feature extraction

In [32]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [36]:
def model_fn():
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
def preprocess(content):
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1697546941232_0001/container_1697546941232_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:403: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [39]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
print("Nombre d'images", features_df.count())
print("Nouvelle taille d'image", np.array(features_df.collect()[0][2]).shape)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nombre d'images 22688
Nouvelle taille d'image (1280,)

In [65]:
to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())
features_df = features_df.withColumn("features_vectors", to_vector("features"))

pca = PCA(k=300, inputCol="features_vectors", outputCol="k=1280").fit(features_df)
print(pca.explainedVariance)
cumValues = pca.explainedVariance.cumsum()

plt.figure(figsize=(10,8))
plt.plot(range(1,300), cumValues, marker = 'o', linestyle='--')
plt.title('Variances')
plt.xlabel('Composantes principales')
plt.ylabel('Variance cumulative')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'plt' is not defined
Traceback (most recent call last):
NameError: name 'plt' is not defined



In [74]:
cumValues

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([0.10140991, 0.18146971, 0.24497275, 0.29512062, 0.3304749 ,
       0.35963482, 0.38736696, 0.41021584, 0.43007528, 0.44916681,
       0.46570388, 0.48036676, 0.4943486 , 0.50803438, 0.52138227,
       0.53388261, 0.54543344, 0.55618266, 0.56598303, 0.5756931 ,
       0.5848514 , 0.59313439, 0.60102211, 0.60852994, 0.61567811,
       0.62275046, 0.62958688, 0.63580523, 0.64192728, 0.64781776,
       0.65352091, 0.65909674, 0.66437059, 0.6694206 , 0.67420584,
       0.67889422, 0.68348949, 0.68783589, 0.69207107, 0.69616314,
       0.7000655 , 0.70395478, 0.70779924, 0.71152294, 0.71515285,
       0.71857505, 0.72197208, 0.72530282, 0.72859338, 0.73183043,
       0.73502978, 0.73808654, 0.74105651, 0.74393957, 0.74676962,
       0.74956232, 0.75231609, 0.75500416, 0.75760228, 0.76012723,
       0.76260927, 0.765061  , 0.76741254, 0.76973088, 0.7720228 ,
       0.77426798, 0.77646268, 0.77863138, 0.78075761, 0.7828701 ,
       0.78493262, 0.78692534, 0.78885698, 0.79076407, 0.79263

In [75]:
pca = PCA(k = 185, 
          inputCol = 'features_vectors', 
          outputCol = 'features_PCA').fit(features_df)

df_pca = pca.transform(features_df)
print('Explained Variance Ratio', pca.explainedVariance.toArray().sum())
df_pca.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
features_pca = df_pca.select('features_PCA').toPandas()
categ = df_pca.select('categ').toPandas()

def expand_row(row):
    list_ = []
    for val in row.values[0]:
        list_.append(val)
    return pd.Series(list_)
    
df_final = features_pca.apply(compute_row, axis=1) 
df_final['Catégorie'] = categ


In [77]:
df_final

In [17]:
#parquet_files = df_final.repartition(24)
#parquet_files.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…